# 路由链
路由链可以根据输入决定到底使用哪条子链。

路由器由两个组件组成：

路由链（Router Chain）：路由器链本身，负责选择要调用的下一个链

目标子链集（destination_chains）：路由器链可以路由到的链

In [14]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain


# 初始化对话模型
llm = ChatOpenAI(
    model="deepseek-r1",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
    streaming=True
)

# 定义多个子链的提示词模版
#第一个提示适合回答物理问题
physics_template = """你是一个非常聪明的物理专家。 \
你擅长用一种简洁并且易于理解的方式去回答问题。\
当你不知道问题的答案时，你承认\
你不知道.

这是一个问题:
{input}"""


#第二个提示适合回答数学问题
math_template = """你是一个非常优秀的数学家。 \
你擅长回答数学问题。 \
你之所以如此优秀， \
是因为你能够将棘手的问题分解为组成部分，\
回答组成部分，然后将它们组合在一起，回答更广泛的问题。

这是一个问题：
{input}"""


#第三个适合回答历史问题
history_template = """你是以为非常优秀的历史学家。 \
你对一系列历史时期的人物、事件和背景有着极好的学识和理解\
你有能力思考、反思、辩证、讨论和评估过去。\
你尊重历史证据，并有能力利用它来支持你的解释和判断。

这是一个问题:
{input}"""


#第四个适合回答计算机问题
computerscience_template = """ 你是一个成功的计算机科学专家。\
你有创造力、协作精神、\
前瞻性思维、自信、解决问题的能力、\
对理论和算法的理解以及出色的沟通技巧。\
你非常擅长回答编程问题。\
你之所以如此优秀，是因为你知道  \
如何通过以机器可以轻松解释的命令式步骤描述解决方案来解决问题，\
并且你知道如何选择在时间复杂性和空间复杂性之间取得良好平衡的解决方案。

这是一个问题:
{input}"""

# 创建提示词集合
prompt_infos = [
    {
        "name": "物理",
        "description": "物理专家",
        "prompt": physics_template
    },
    {
        "name": "数学",
        "description": "数学专家",
        "prompt": math_template
    },
    {
        "name": "历史",
        "description": "历史专家",
        "prompt": history_template
    },
    {
        "name": "计算机科学",
        "description": "计算机科学专家",
        "prompt": computerscience_template
    }
]


# 生成多个子链
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt = p_info["prompt"]
    prompt_template = ChatPromptTemplate.from_template(template=prompt)
    chain = LLMChain(
        llm=llm,
        prompt=prompt_template
    )
    destination_chains[name] = chain

# 子链字典的描述
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# 创建一个默认的链
default_template = """
你是一个智能的AI助手。\
用一句话回答用户的问题。\
问题如下：
{input}
"""
default_prompt = ChatPromptTemplate.from_template(template=default_template)
default_chain = LLMChain(
    llm=llm,
    prompt=default_prompt
)

# 多提示路由模版
MULTI_PROMPT_ROUTER_TEMPLATE  = """
给语言模型一个原始文本输入，\
让其选择最适合输入的模型提示。\
系统将为您提供可用提示的名称以及最适合改提示的描述。\
如果你认为修改原始输入最终会导致语言模型做出更好的响应，\
你也可以修改原始输入。


<< 格式 >>
返回一个带有JSON对象的markdown代码片段，该JSON对象的格式如下：
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}



记住：“destination”必须是下面指定的候选提示名称之一，\
或者如果输入不太适合任何候选提示，\
则可以是 “DEFAULT” 。
记住：如果您认为不需要任何修改，\
则 “next_inputs” 可以只是原始输入。

<< 候选提示 >>
{destinations}

<< 输入 >>
{{input}}

<< 输出 (记得要包含 ```json)>>

样例:
<< 输入 >>
"什么是黑体辐射?"
<< 输出 >>
```json
{{{{
    "destination": 字符串 \ 使用的提示名字或者使用 "DEFAULT"
    "next_inputs": 字符串 \ 原始输入的改进版本
}}}}

"""

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# 构建路由提示词
router_prompt = PromptTemplate.from_template(
    template=router_template,
    output_parser=RouterOutputParser()
)

# 构建路由链
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt
)

# 构建多提示路由链
final_chain = MultiPromptChain(
    router_chain=router_chain,# 路由链
    destination_chains=destination_chains,# 目标子链
    default_chain=default_chain,# 默认链
    verbose=True # 是否打印详细信息
)

# 测试多提示路由链
for chunk in final_chain.stream({"input": "什么是黑体辐射?"}):
    print(chunk,end="",flush=True)

for chunk in final_chain.stream({"input": "2+2等于多少？"}):
    print(chunk,end="",flush=True)

for chunk in final_chain.stream({"input": "中国历史上最伟大的皇帝是谁？"}):
    print(chunk,end="",flush=True)
    
for chunk in final_chain.stream({"input": "简单介绍一下计算机科学的发展历程"}):
    print(chunk,end="",flush=True)

for chunk in final_chain.stream({"input": "为什么我们身体里的每个细胞都包含DNA？"}):
    print(chunk,end="",flush=True)






<>:117: SyntaxWarning: invalid escape sequence '\ '
<>:117: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Administrator\AppData\Local\Temp\ipykernel_38072\2024827929.py:117: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE  = """




> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑体辐射?'}
> Finished chain.
{'input': '什么是黑体辐射?', 'text': '**黑体辐射**是指一个理想化的物体（称为**黑体**）在热平衡状态下发出的电磁辐射。黑体能**完全吸收**所有入射的电磁波（不反射、不透过），同时又能以特定波长**发射辐射**，其辐射特性仅取决于温度。\n\n---\n\n### **关键点：**\n1. **黑体的重要性**  \n   黑体是一个理论模型（如密闭空腔上的小孔近似黑体），但它揭示了热辐射的普遍规律。恒星（如太阳）、高温金属，甚至宇宙微波背景辐射都近似符合黑体辐射特性。\n\n2. **经典物理的失败**  \n   - 19世纪末，经典理论（如瑞利-金斯公式）预测短波（紫外光）辐射能量会趋向无穷大（“**紫外灾变**”），与实验严重矛盾。\n   - 维恩公式在短波区吻合实验，但长波区失效。\n\n3. **普朗克的突破（1900年）**  \n   普朗克提出**能量量子化**假说：黑体辐射的能量以离散的“能量包”（量子）形式发射，每份能量为 \\( E = h\\nu \\)（\\( h \\) 是普朗克常数，\\( \\nu \\) 是频率）。由此导出的**普朗克公式**完美拟合实验数据，**开创了量子力学**。\n\n4. **黑体辐射的规律**  \n   - **维恩位移定律**：峰值波长 \\( \\lambda_{\\text{max}} \\propto 1/T \\)（温度越高，辐射峰值波长越短，如烧红的铁块随温度升高从红变白）。\n   - **斯特藩-玻尔兹曼定律**：总辐射功率 \\( P \\propto T^4 \\)。\n\n---\n\n### **意义：**\n- **量子力学的诞生**：普朗克的量子假说颠覆了经典物理中能量连续的观念。\n- **应用**：用于测量恒星温度、红外热成像、宇宙学（宇宙微波背景辐射的温度测定）等。\n\n简而言之，黑体辐射是理解热与光本质的基石，也是量子革命的第一声惊雷 🌟。'}

> Entering new MultiPromptChain chain...
数学: {'input': '2+2